# torchvision.datasets.UFC101の使い方

torchvisionのdatasetを使ってUFC101を読み込む．

- https://pytorch.org/vision/stable/datasets.html?highlight=ucf101#torchvision.datasets.UCF101


# データセットの準備

UFC101はあらかじめダウンロードして展開済みであるとする．
- `/dataset/UCF101/video/`以下に，101クラスのサブディレクトリがある
- `/dataset/UCF101/ucfTrainTestlist/`以下に，UCF101のアノテーションファイルであるtrainlist0{1,2,3}.txtなどがある

## フォルダ構成

```bash
$ tree -I "*.avi" /dataset/UCF101
/dataset/UCF101
├── ucfTrainTestlist
│   ├── classInd.txt
│   ├── testlist01.txt
│   ├── testlist02.txt
│   ├── testlist03.txt
│   ├── trainlist01.txt
│   ├── trainlist02.txt
│   └── trainlist03.txt
└── video
    ├── ApplyEyeMakeup
    ├── ApplyLipstick
    ├── Archery
    ├── BabyCrawling
    ├── BalanceBeam
...
    ├── Typing
    ├── UnevenBars
    ├── VolleyballSpiking
    ├── WalkingWithDog
    ├── WallPushups
    ├── WritingOnBoard
    └── YoYo

$ head -5 /dataset/UCF101/ucfTrainTestlist/*
==> /dataset/UCF101/ucfTrainTestlist/classInd.txt <==
1 ApplyEyeMakeup
2 ApplyLipstick
3 Archery
4 BabyCrawling
5 BalanceBeam

==> /dataset/UCF101/ucfTrainTestlist/testlist01.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi

==> /dataset/UCF101/ucfTrainTestlist/testlist02.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi

==> /dataset/UCF101/ucfTrainTestlist/testlist03.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c01.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c02.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c03.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c04.avi
ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c05.avi

==> /dataset/UCF101/ucfTrainTestlist/trainlist01.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1

==> /dataset/UCF101/ucfTrainTestlist/trainlist02.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi 1

==> /dataset/UCF101/ucfTrainTestlist/trainlist03.txt <==
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi 1
ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi 1
```


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

from torchvision.models import resnet18
from torchvision import transforms
from torchvision.datasets import UCF101

import torchinfo

from tqdm.notebook import tqdm

import os
import pickle

argparseを真似たパラメータ設定．
- rootで指定したディレクトリには，101クラスのサブディレクトリがあること
- annotation_pathには，UCF101のアノテーションファイルであるtrainlist0{1,2,3}.txtなどがあること

In [2]:
class Args:
    def __init__(self):
        self.metadata_path = '/dataset/UCF101/'
        self.root = self.metadata_path + 'video/'
        self.annotation_path = \
            self.metadata_path + 'ucfTrainTestlist/'
        self.frames_per_clip = 8
        self.step_between_clips = 8
        self.batch_size = 8
        self.num_workers = 1


args = Args()


## transform

- UCF101を読み込むとuint8なので，255で割ってfloatにする．
- torchvisionのUCF101データセットは(T, H, W, C)の形式．しかしpytorchvideoのx3dの入力形式は(B, C, T, H, W)らしいので，それに合わせる．

- 画像を224x224にリサイズする．torchvision.transforms.Resizeはshapeが`[..., H, W]`ならOKなので，画像だけでなく動画もOK．Cropなども同様．
https://github.com/pytorch/vision/blob/183a722169421c83638e68ee2d8fc5bd3415c4b4/torchvision/transforms/transforms.py#L227

```text
    If the image is torch Tensor, it is expected
    to have [..., H, W] shape, where ... means an arbitrary number of leading dimensions
```

In [3]:
transform = transforms.Compose([
    transforms.Lambda(lambda x: x / 255.),

    # (T, H, W, C) --> (C, T, H, W)
    transforms.Lambda(lambda x: x.permute(3, 0, 1, 2)),

    transforms.Resize(224),
])

データセットはimage, audio, labelの三組を返す．

しかしUCF101には音声がない動画が約半数．
- 音声あり: 6837
- 音声なし: 6483

そのためdatasetをそのまま使うと，dataloaderで「バッチにできない」というエラーが出てしまう（audioの次元数がサンプルによって異なるため）．

そこでカスタムcollateでaudioを取り除く．取り除いたら`default_collate`に渡す．

参考：https://www.kaggle.com/pevogam/starter-ucf101-with-pytorch

In [4]:
def remove_audio_collate(batch):
    '''
    remove audio channel because
    not all of UCF101 vidoes have audio channel
    '''
    video_only_batch = []
    for video, audio, label in batch:
        video_only_batch.append((video, label))
    return default_collate(video_only_batch)

custom_collate = remove_audio_collate

# メタデータの準備

torchvision.datasets.UCF101は，全動画をスキャンして，FPSなどのメタデータ情報を取得するらしい．
これにかなり時間がかかる（4-5分）．
何もしないと，毎回スキャンするため，時間の無駄になる．
そこで，メタデータを保存して再利用することにする．

## 方針

コードを見たところ，foldやtrainには無関係で，fpcとsbcにだけ依存するらしい．

- https://github.com/pytorch/vision/blob/183a722169421c83638e68ee2d8fc5bd3415c4b4/torchvision/datasets/ucf101.py#L60

```python
        video_clips = VideoClips(
            video_list,
            frames_per_clip,
            step_between_clips,
            frame_rate,
            _precomputed_metadata,
            num_workers=num_workers,
            _video_width=_video_width,
            _video_height=_video_height,
            _video_min_dimension=_video_min_dimension,
            _audio_samples=_audio_samples,
        )
```

上の`VideoClip`が呼び出されたら，`_precomputed_metadata`がNoneならメタデータを作成して，
`.metadata`属性に保存している．

そこで，fpcとsbcををファイル名につけて保存する．

In [5]:
metadata_filename = os.path.join(
    args.metadata_path,
    'UCF101metadata_fpc{}_sbc{}.pickle'.format(
        args.frames_per_clip,
        args.step_between_clips))

if not os.path.exists(metadata_filename):
    # if no metadata, precompute and save metadata
    dataset_dict = UCF101(
        root=args.root,
        annotation_path=args.annotation_path,
        frames_per_clip=args.frames_per_clip,
        step_between_clips=args.step_between_clips,
        num_workers=args.num_workers,
    )
    # now metadata is stored in dataset_dict.metadata

    with open(metadata_filename, "wb") as f:
        pickle.dump(dataset_dict.metadata, f)

with open(metadata_filename, 'rb') as f:
    metadata = pickle.load(f)


## メタデータの注意点

metadataはdict形式で，キー`video_paths`に動画ファイルへのパスを保持している．
そのため動画ファイルのパスが変わったら，metadataを作成し直す（もしくは`video_paths`の中身を書き換える）必要がある．



In [6]:
type(metadata)

dict

In [7]:
metadata.keys()

dict_keys(['video_paths', 'video_pts', 'video_fps'])

In [8]:
metadata['video_paths'][:3]

['/dataset/UCF101/video/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi',
 '/dataset/UCF101/video/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi',
 '/dataset/UCF101/video/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi']

In [9]:
metadata['video_pts'][:3]

[tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
          15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
          29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
          43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
          57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
          71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
          85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
          99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
         113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
         127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
         141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
         155, 156, 157, 158, 159, 160, 161, 162, 163, 164]),
 tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  

In [10]:
metadata['video_fps'][:3]

[25.0, 25.0, 25.0]

# UCF101データセットオブジェクトの作成

- frames_per_clip：1クリップに使うフレーム数
- step_between_clips：次のクリップを開始するフレーム間隔．指定しなければframes_per_clipと同じ，つまりclip間でフレームの重複はない．frames_per_clipsよりも小さくすると，clip間でフレームが重複する（同じフレームが複数のclipで使われる）．frames_per_clipよりも大きくする
- fold：UCF101には3つのスプリットがあるので，1, 2, 3のいずれかを指定する．（論文用の最終結果には，3つのスプリットの平均が報告される）
- _precomputed_metadata：先ほど読み込んだmetadataオブジェクトを指定（ファイル名ではないので，あらかじめファイルから読み込んでおく必要がある）


In [11]:
train_set = UCF101(
    root=args.root,
    annotation_path=args.annotation_path,
    frames_per_clip=args.frames_per_clip,
    step_between_clips=args.step_between_clips,
    fold=1,
    train=True,
    transform=transform,
    _precomputed_metadata=metadata)

val_set = UCF101(
    root=args.root,
    annotation_path=args.annotation_path,
    frames_per_clip=args.frames_per_clip,
    step_between_clips=args.step_between_clips,
    fold=1,
    train=False,
    transform=transform,
    _precomputed_metadata=metadata)

n_classes = 101


データローダーの作成．カスタムcollateをここで指定．

In [12]:
train_loader = DataLoader(
    train_set,
    batch_size=args.batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=custom_collate,
    num_workers=args.num_workers)
    
val_loader = DataLoader(
    val_set,
    batch_size=args.batch_size,
    shuffle=False,
    drop_last=True,
    collate_fn=custom_collate,
    num_workers=args.num_workers)


# dataloaderの挙動

バッチを取り出して挙動を確認．
- train_loaderでは`shuffle=True`にしているので，ランダムなラベルが得られている．
- val_loaderでは`shuffle=False`なので，最初はラベル0のclipばかり


## エラーが発生する場合

frames_per_clipを16に設定しているのに17枚ある，というエラーが発生する場合がある．

```
Original Traceback (most recent call last):
File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
data = fetcher.fetch(index)
File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
data = [self.dataset[idx] for idx in possibly_batched_index]
File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in 
data = [self.dataset[idx] for idx in possibly_batched_index]
File "/usr/local/lib/python3.9/dist-packages/torchvision/datasets/ucf101.py", line 102, in getitem
video, audio, info, video_idx = self.video_clips.get_clip(idx)
File "/usr/local/lib/python3.9/dist-packages/torchvision/datasets/video_utils.py", line 382, in get_clip
assert len(video) == self.num_frames, "{} x {}".format(
AssertionError: torch.Size([17, 224, 224, 3]) x 8
```

これはtorchvisionのバグのようなので，バージョンを変えてみたほうがよい．
- 発生しない：torch==1.8.1+cpu torchvision==0.9.1+cpu
- 発生する：torch==1.9.0+cpu torchvision==0.10.0+cpu

0.9.1ではtorchvision.io.videoのワーニングが出るが，0.10.0では出ないので，ptsからsecに変更された影響かもしれない

```
/usr/local/lib/python3.9/site-packages/torchvision/io/video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
```

参考情報：
- UCF101: Dataloader Fail on assertion #4112 https://github.com/pytorch/vision/issues/4112
- VideoClips Assertion Error #1884 https://github.com/pytorch/vision/issues/1884
- VideoClips Assertion Error https://gitmemory.com/issue/pytorch/vision/1884/595211331




In [13]:

# # torchvisionのvideo.pyで，ワーニングが多数出るのでそれを抑制するなら
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning,
#                                    module='torchvision')

print('train_loader')
for i, (data, label) in enumerate(train_loader):
    print('batch {}:'.format(i), label.cpu().numpy())
    if i > 5:
        break

print('val_loader')
for i, (data, label) in enumerate(val_loader):
    print('batch {}:'.format(i), label.cpu().numpy())
    if i > 5:
        break

train_loader
/usr/local/lib/python3.9/site-packages/torchvision/io/video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  warnings.warn(
batch 0: [41 36  2 45 31 20 69 92]
batch 1: [ 55  88  46  75  48  38 100  33]
batch 2: [44 54 35 35  1 95 61 34]
batch 3: [68 10 92 77 33 60 17 45]
batch 4: [54 50 68 75 24 62 43 68]
batch 5: [14 49 70 26 43 64 60 59]
batch 6: [46 83 49 71  6 22 82 47]
val_loader
/usr/local/lib/python3.9/site-packages/torchvision/io/video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  warnings.warn(
batch 0: [0 0 0 0 0 0 0 0]
batch 1: [0 0 0 0 0 0 0 0]
batch 2: [0 0 0 0 0 0 0 0]
batch 3: [0 0 0 0 0 0 0 0]
batch 4: [0 0 0 0 0 0 0 0]
batch 5: [0 0 0 0 0 0 0 0]
batch 6: [0 0 0 0 0 0 0 0]


データローダーのlenを確認する．

- 学習用ビデオ数は9000程度のはずなのに，train_setのlengthは非常に多い
  - おそらく，各ビデオからサンプリングしたclip数になっている
      - 例えば，16フレームのclipを，16フレーム毎にサンプルすると，160フレームの動画からは10個のclipがサンプリングされる
  - 各ビデオから同じ数のクリップがサンプルされているとは限らない（確認できていないが，おそらくそう）

In [14]:
print('train set')
print('num batches', len(train_loader))
print('num samples', len(train_set))
print('num samples / batch_size = ', len(train_set) / args.batch_size)

train set
num batches 27389
num samples 219119
num samples / batch_size =  27389.875


In [15]:
print('val set')
print('num batches', len(val_loader))
print('num samples', len(val_set))
print('num samples / batch_size = ', len(val_set) / args.batch_size)

val set
num batches 10690
num samples 85526
num samples / batch_size =  10690.75


# 学習ループ

学習ループを想定して，dataloaderからバッチを取り出す

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
num_epochs = 2

for epoch in range(num_epochs):
    print("Epoch ", epoch)

    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device)
        # current batch size may vary at the end of the epoch
        #  if drop_last=False for data loaders
        bs = inputs.size(0)

        print('batch ', batch_idx)
        print('batch size', bs)
        print('input shape', inputs.shape)
        print('target shape', targets.shape)

        if batch_idx > 2:
            break  # stop for demonstration

Epoch  0
/usr/local/lib/python3.9/site-packages/torchvision/io/video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  warnings.warn(
batch  0
batch size 8
input shape torch.Size([8, 3, 8, 224, 298])
target shape torch.Size([8])
batch  1
batch size 8
input shape torch.Size([8, 3, 8, 224, 298])
target shape torch.Size([8])
batch  2
batch size 8
input shape torch.Size([8, 3, 8, 224, 298])
target shape torch.Size([8])
batch  3
batch size 8
input shape torch.Size([8, 3, 8, 224, 298])
target shape torch.Size([8])
Epoch  1
/usr/local/lib/python3.9/site-packages/torchvision/io/video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  warnings.warn(
batch  0
batch size 8
input shape torch.Size([8, 3, 8, 224, 298])
target shape torch.Size([8])
batch  1
batch size 8
input shape torch.Size([8, 3, 8, 224, 298])
target shape torch.Size([